In [13]:
import pandas as pd
import zipfile
import os, shutil, random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

In [2]:
zip_path = "/content/drive/MyDrive/archive (1).zip"
extract_path = "/content/car_vs_bike"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
  zip_ref.extractall(extract_path)

In [3]:
dataset_dir = "/content/car_vs_bike/Car-Bike-Dataset"
output_dir = "/content/car_bike_split"

classes = ["Car","Bike"]

for cls in classes:
  src = os.path.join(dataset_dir,cls)
  imgs = os.listdir(src)
  random.shuffle(imgs)

  split1 = int(0.8 * len(imgs))

  train_imgs = imgs[:split1]
  test_imgs = imgs[split1:]

  os.makedirs(os.path.join(output_dir, "train", cls), exist_ok=True)
  os.makedirs(os.path.join(output_dir, "test", cls), exist_ok=True)

  for img in train_imgs:
    shutil.copy(os.path.join(src, img), os.path.join(output_dir, "train", cls))
  for img in test_imgs:
    shutil.copy(os.path.join(src, img), os.path.join(output_dir, "test", cls))


In [5]:
import os

base_dir = "car_bike_split"

for split in ["train", "test"]:
    print(f"--- {split.upper()} ---")
    for cls in ["Car", "Bike"]:
        folder = os.path.join(base_dir, split, cls)
        count = len(os.listdir(folder))
        print(f"{cls}: {count} images")


--- TRAIN ---
Car: 1934 images
Bike: 1908 images
--- TEST ---
Car: 734 images
Bike: 708 images


In [8]:
train_dir = "/content/car_bike_split/train"
test_dir = "/content/car_bike_split/test"

# Data Augmentation for train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir, target_size=(224,224), batch_size=32, class_mode='binary')
test_gen = test_datagen.flow_from_directory(test_dir, target_size=(224,224), batch_size=32, class_mode='binary')


Found 3842 images belonging to 2 classes.
Found 1442 images belonging to 2 classes.


In [14]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze base model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [15]:
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=5
)


Epoch 1/5
 62/121 ━━━━━━━━━━━━━━━━━━━━ 1:38 2s/step - accuracy: 0.8027 - loss: 0.4326

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


121/121 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.8633 - loss: 0.3190 - val_accuracy: 0.9924 - val_loss: 0.0374
Epoch 2/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9840 - loss: 0.0569 - val_accuracy: 0.9931 - val_loss: 0.0260
Epoch 3/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.9858 - loss: 0.0441 - val_accuracy: 0.9924 - val_loss: 0.0226
Epoch 4/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.9895 - loss: 0.0366 - val_accuracy: 0.9931 - val_loss: 0.0201
Epoch 5/5
121/121 ━━━━━━━━━━━━━━━━━━━━ 253s 2s/step - accuracy: 0.9878 - loss: 0.0395 - val_accuracy: 0.9938 - val_loss: 0.0193


In [16]:
model.save("vehicle_classifier_mobilenet.keras")